In [83]:
def load_graph_from_mtx(filename):
    graph = {}
    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            # preskocimo prva tri reda (jer su tu podaci o broju grana, cvorova...)
            if i < 3:
                continue
            
            nodes = line.strip().split()
            node1, node2 = int(nodes[0]), int(nodes[1])

            if node1 not in graph:
                graph[node1] = set()
            if node2 not in graph:
                graph[node2] = set()

            graph[node1].add(node2)
            graph[node2].add(node1)

    return graph


In [84]:
from collections import deque
import math

filename = "grafovi/manji/karate.mtx"

graph = load_graph_from_mtx(filename)
from collections import deque
import math

def all_pairs_distance_matrix(graph):
    """
    Calculate all-pairs shortest distances using BFS for each node.
    :param graph: dict where keys are nodes (1 to n) and values are lists of neighbors
    :return: 2D list (matrix) where element [i][j] is the shortest distance from node i+1 to j+1
    """
    nodes = sorted(graph.keys())  # Sortiraj čvorove kako bi odgovarali indeksima matrice
    n = len(nodes)
    distance_matrix = [[math.inf] * n for _ in range(n)]  # Inicijalizuj matricu sa beskonačnostima

    for start in nodes:
        visited = {node: False for node in nodes}
        d_local = {node: math.inf for node in nodes}
        q = deque()
        d_local[start] = 0
        visited[start] = True
        q.append(start)

        while q:
            current = q.popleft()
            for neighbor in graph.get(current, []):
                if not visited[neighbor]:
                    visited[neighbor] = True
                    q.append(neighbor)
                    d_local[neighbor] = d_local[current] + 1

        # Popuni red u matrici udaljenosti za čvor `start`
        for node in nodes:
            distance_matrix[start - 1][node - 1] = d_local[node]

    return distance_matrix

distance_matrix = all_pairs_distance_matrix(graph)

for row in distance_matrix:
    print(row)
    
def valid_burning_sequence(s, n, d):
    """
    Validate if the given burning sequence is valid for a graph of size n.
    :param s: List of burning nodes (sequence of nodes).
    :param n: Number of nodes in the graph.
    :param d: Distance matrix (2D list) where d[i][j] represents the shortest distance between node i+1 and j+1.
    :return: True if the sequence is valid, otherwise False.
    """
    counter = 0

    for i in range(n):
        b = len(s)
        for j in range(b):
            if d[i][s[j] - 1] <= b - (j + 1):  # Adjust index for 1-based node numbering
                counter += 1
                break

    return counter == n

def BFF(n, graph, d):
    """
    Burning Farthest-First algorithm to find the burning sequence for the graph.
    :param n: Number of nodes in the graph.
    :param graph: Graph represented as a dictionary (adjacency list).
    :param d: Distance matrix (2D list).
    :return: List representing the burning sequence.
    """
    first_vertex = 0
    s = [first_vertex]  # Burning sequence
    q = [0]  # Queue for BFS, starting with the first vertex
    dist = [d[i][0] for i in range(n)]  # Initial distances from the first vertex
    C = [0] * n  # Burned status (0 = not burned, 1 = burned)
    C[0] = 1
    number_of_burned_vertices = 1

    while number_of_burned_vertices < n:
        # Process the queue for the current burning stage
        q_size = len(q)
        for _ in range(q_size):
            v = q.pop(0)
            for neighbor in graph.get(v + 1, []):  # Adjust for 1-based indexing
                neighbor_idx = neighbor - 1  # Convert to 0-based indexing
                if C[neighbor_idx] == 0:
                    q.append(neighbor_idx)
                    C[neighbor_idx] = 1
                    number_of_burned_vertices += 1

        # Find the farthest unburned vertex
        farthest_vertex = 0
        larger_distance = 0
        for i in range(n):
            if C[i] == 0 and dist[i] > larger_distance:
                larger_distance = dist[i]
                farthest_vertex = i

        # Burn the farthest vertex
        q.append(farthest_vertex)
        s.append(farthest_vertex)
        C[farthest_vertex] = 1
        number_of_burned_vertices += 1

        # Update distances after burning the new vertex
        for i in range(n):
            dist[i] = min(dist[i], d[i][farthest_vertex])

    return [v + 1 for v in s]  # Convert back to 1-based indexing


[0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 3, 2, 1, 3, 1, 3, 1, 3, 3, 2, 2, 3, 2, 2, 3, 2, 1, 2, 2]
[1, 0, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 3, 3, 3, 1, 3, 1, 3, 1, 3, 3, 3, 3, 3, 2, 2, 3, 1, 2, 2, 2]
[1, 1, 0, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 1, 1, 2, 2, 2, 1, 2]
[1, 1, 1, 0, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1, 3, 3, 3, 2, 3, 2, 3, 2, 3, 3, 3, 3, 3, 2, 2, 3, 2, 2, 2, 2]
[1, 2, 2, 2, 0, 2, 1, 2, 2, 3, 1, 2, 2, 2, 4, 4, 2, 2, 4, 2, 4, 2, 4, 4, 3, 3, 4, 3, 3, 4, 3, 2, 3, 3]
[1, 2, 2, 2, 2, 0, 1, 2, 2, 3, 1, 2, 2, 2, 4, 4, 1, 2, 4, 2, 4, 2, 4, 4, 3, 3, 4, 3, 3, 4, 3, 2, 3, 3]
[1, 2, 2, 2, 1, 1, 0, 2, 2, 3, 2, 2, 2, 2, 4, 4, 1, 2, 4, 2, 4, 2, 4, 4, 3, 3, 4, 3, 3, 4, 3, 2, 3, 3]
[1, 1, 1, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 3, 3, 3, 2, 3, 2, 3, 2, 3, 3, 3, 3, 4, 2, 2, 3, 2, 2, 2, 3]
[1, 2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 1, 2, 1, 1]
[2, 2, 1, 2, 3, 3, 3, 2, 2, 0, 3, 3, 3, 2, 2, 2, 4, 3, 2, 2, 2, 3, 2, 2, 

In [85]:
def Gr(n, k, d):
    s = []
    max_value = 0
    max_value_vertex = 0
    C = [0] * (n + 1)
    
    # Initialize adjacency lists
    A = [[] for _ in range(n + 1)]
    for i in range(1, n + 1):
        a = [j for j in range(1, n + 1) if d[i - 1][j - 1] <= k - 1]
        A[i] = a
    
    # Main loop
    for r in range(k - 1, -1, -1):
        if r < k - 1:
            for i in range(1, n + 1):
                j = 0
                while j < len(A[i]):
                    if d[i - 1][A[i][j] - 1] > r or C[A[i][j]] == 1:
                        A[i][j] = A[i][-1]
                        A[i].pop()
                    else:
                        j += 1
        max_value = 0
        max_value_vertex = 0
        for i in range(1, n + 1):
            if len(A[i]) > max_value:
                max_value = len(A[i])
                max_value_vertex = i
        
        for j in range(len(A[max_value_vertex])):
            C[A[max_value_vertex][j]] = 1
        
        s.append(max_value_vertex)
    
    return s


In [86]:
import math 
d = all_pairs_distance_matrix(graph)
print(len(d))
n = len(graph)
burning_sequence = BFF(n, graph, d)
low = math.ceil((len(burning_sequence))+2)/3
high = len(burning_sequence) - 1
best_burning_sequence_size = len(burning_sequence)
print("Burning sequence:", burning_sequence)

k = math.floor((high+low)/2)+1
s = Gr(n, k, d)
   
print(valid_burning_sequence([4, 34, 6, 25, 5, 12, 18, 22], n, d))


print("Gr burning sequence:",s)

34
Burning sequence: [1, 15, 16, 1]
True
Gr burning sequence: [32, 6, 0]
